In [ ]:
# ============================================================================
# WEATHER IMPACT EMAIL NOTIFIER
# Parametric Insurance Demo — Notebook 4
# ============================================================================
#
# Architecture:
#   Event Grid (policy.weather.impact)
#       │
#       ▼
#   [This notebook, triggered on schedule or manually]
#       │
#       ▼
#   Orchestrator Agent (New Foundry — Responses API)
#       │  uses Fabric Data Agent as a tool
#       │  to query: policy details, prior claims, weather context
#       │
#       ▼
#   Fabric Data Agent → lakehouse SQL query → structured context
#       │
#       ▼
#   Orchestrator Agent composes professional email
#       │
#       ▼
#   Email records saved to Delta table (email_notifications)
#   [Optional] Send via Logic App webhook / SendGrid
#
# Prerequisites:
#   - weather_alert_policy_impact.ipynb must have populated weather_impact_events
#   - A Fabric Data Agent must be created in your workspace and connected
#     to the parametric_insurance_lakehouse
#   - An Orchestrator Agent (Foundry Responses API) must be created and
#     configured with the Fabric Data Agent as a registered tool/action
# ============================================================================

# 📧 Weather Impact Email Notifier

```
weather_impact_events (Delta)
        │
        ▼
  Orchestrator Agent  ──── tool call ────►  Fabric Data Agent
        │                                        │
        │                                  Lakehouse SQL query
        │◄───────── enriched context ────────────┘
        │
   compose email
        │
        ▼
  email_notifications (Delta) + [optional webhook send]
```

| Step | Description | Agent |
|------|-------------|-------|
| 0 | Config & imports | — |
| 1 | Load unnotified impact events | Spark SQL |
| 2 | Orchestrator queries Fabric Data Agent | **Orchestrator Agent** → **Fabric Data Agent** |
| 3 | Orchestrator composes professional email | **Orchestrator Agent** |
| 4 | Persist emails to Delta | Spark |
| 5 | [Optional] Dispatch via webhook | HTTP |
| 6 | Mark impacts as notified | Delta merge |
| 7 | Summary dashboard | Spark SQL |

---
## 🔧 Step 0 — Configuration & Imports

In [ ]:
%pip install azure-ai-projects==2.0.0b3 azure-core azure-ai-agents --quiet

In [ ]:
import os
import json
import uuid
import time
import requests
import warnings
import textwrap
import notebookutils
from datetime import datetime, timedelta, timezone
from dataclasses import dataclass
from typing import List, Dict, Any, Optional
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, TimestampType, BooleanType
)
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

warnings.filterwarnings('ignore')

# ---------------------------------------------------------------------------
# CONFIGURATION — Edit these values for your environment
# ---------------------------------------------------------------------------

@dataclass
class EmailNotifierConfig:
    """Configuration for the weather impact email notifier."""

    # ── Foundry / AI Project (New experience — Responses API) ──────────────
    # Endpoint format: https://<resource>.services.ai.azure.com/api/projects/<project>
    foundry_endpoint:        str = "<foundry-endpoint>"

    # The Orchestrator Agent is responsible for:
    #   1. Calling the Fabric Data Agent to retrieve enriched policyholder context
    #   2. Composing a professional email using that context
    orchestrator_agent_id:   str = "<orchestrator-agent-id>"

    # The Fabric Data Agent is connected to the parametric_insurance_lakehouse.
    # It accepts natural-language questions and returns structured query results.
    # Register it as a "connected agent" action inside the Orchestrator Agent.
    fabric_data_agent_id:    str = "<fabric-data-agent-id>"

    # ── Processing limits ──────────────────────────────────────────────────
    # Maximum number of impact events to process in a single run.
    # Set to a low number (3–5) for live demo; higher for batch processing.
    max_events_per_run:      int = 10

    # Only process impacts with a risk score at or above this threshold.
    min_risk_score:          float = 0.3

    # ── Optional email dispatch ────────────────────────────────────────────
    # Leave blank to store emails in Delta only (no actual sending).
    # Set to a Logic App / Power Automate HTTP trigger URL to send emails.
    email_dispatch_webhook:  str = ""

    # ── Insurance company branding ─────────────────────────────────────────
    insurer_name:            str = "Parametric Shield Insurance"
    insurer_support_email:   str = "support@parametricshield.com"
    insurer_support_phone:   str = "1-800-555-0199"
    insurer_portal_url:      str = "https://portal.parametricshield.com"


config = EmailNotifierConfig()

# Override from Variable Library
try:
    env_lib = notebookutils.variableLibrary.getLibrary("environmentVariables")
    config.foundry_endpoint      = getattr(env_lib, "FOUNDRY_ENDPOINT",         config.foundry_endpoint)
    config.orchestrator_agent_id = getattr(env_lib, "ORCHESTRATOR_AGENT_ID",    config.orchestrator_agent_id)
    config.fabric_data_agent_id  = getattr(env_lib, "FABRIC_DATA_AGENT_ID",     config.fabric_data_agent_id)
    config.email_dispatch_webhook= getattr(env_lib, "EMAIL_DISPATCH_WEBHOOK",   config.email_dispatch_webhook)
except Exception:
    pass

# Widget overrides
try:
    config.foundry_endpoint      = config.foundry_endpoint      or notebookutils.mssparkutils.widgets.get("foundry_endpoint")
    config.orchestrator_agent_id = config.orchestrator_agent_id or notebookutils.mssparkutils.widgets.get("orchestrator_agent_id")
    config.fabric_data_agent_id  = config.fabric_data_agent_id  or notebookutils.mssparkutils.widgets.get("fabric_data_agent_id")
except Exception:
    pass

FOUNDRY_ENABLED = bool(
    config.foundry_endpoint and
    config.orchestrator_agent_id and
    config.foundry_endpoint != "<foundry-endpoint>"
)

FABRIC_DATA_AGENT_ENABLED = bool(
    config.fabric_data_agent_id and
    config.fabric_data_agent_id != "<fabric-data-agent-id>"
)

DISPATCH_ENABLED = bool(config.email_dispatch_webhook)

spark    = SparkSession.builder.getOrCreate()
RUN_ID   = uuid.uuid4().hex[:8].upper()
now_utc  = datetime.now(timezone.utc).replace(tzinfo=None)

print("📧 Weather Impact Email Notifier")
print(f"   Run ID:                {RUN_ID}")
print(f"   Timestamp (UTC):       {now_utc.isoformat()}Z")
print(f"   Foundry Agent:         {'✅ ENABLED' if FOUNDRY_ENABLED else '⚠️  DISABLED (template fallback)'}")
print(f"   Fabric Data Agent:     {'✅ ENABLED' if FABRIC_DATA_AGENT_ENABLED else '⚠️  DISABLED (event data only)'}")
print(f"   Email Dispatch:        {'✅ ENABLED (webhook)' if DISPATCH_ENABLED else '📋 LOCAL ONLY (Delta table)'}")
print(f"   Max events/run:        {config.max_events_per_run}")
print(f"   Min risk score:        {config.min_risk_score}")

---
## 🏗️ Setup: Foundry Client + Email Schema

In [ ]:
# ============================================================================
# Foundry AI Project Client (New Responses API)
# ============================================================================

os.environ["AZURE_CLIENT_ID"]     = ""
os.environ["AZURE_TENANT_ID"]     = ""
os.environ["AZURE_CLIENT_SECRET"] = ""

try:
    env_lib = notebookutils.variableLibrary.getLibrary("environmentVariables")
    os.environ["AZURE_CLIENT_ID"]     = env_lib.AZURE_CLIENT_ID
    os.environ["AZURE_TENANT_ID"]     = env_lib.AZURE_TENANT_ID
    os.environ["AZURE_CLIENT_SECRET"] = env_lib.AZURE_CLIENT_SECRET
except Exception:
    pass

foundry_client: Optional[AIProjectClient] = None
orchestrator_agent = None

if FOUNDRY_ENABLED:
    try:
        foundry_client = AIProjectClient(
            endpoint   = config.foundry_endpoint,
            credential = DefaultAzureCredential(),
        )
        # Retrieve the pre-configured Orchestrator Agent
        orchestrator_agent = foundry_client.agents.get_agent(config.orchestrator_agent_id)
        print(f"✅ Connected to Foundry. Orchestrator Agent: '{orchestrator_agent.name}'")

        if FABRIC_DATA_AGENT_ENABLED:
            try:
                fabric_agent = foundry_client.agents.get_agent(config.fabric_data_agent_id)
                print(f"✅ Fabric Data Agent registered: '{fabric_agent.name}'")
                print("   The Orchestrator will call this agent as a tool to query the lakehouse.")
            except Exception as e:
                print(f"⚠️  Fabric Data Agent not found by ID '{config.fabric_data_agent_id}': {e}")
                print("   Orchestrator will use event data only (no lakehouse enrichment).")
    except Exception as e:
        print(f"⚠️  Could not initialise Foundry client: {e}")
        print("   Falling back to rule-based email template.")
        foundry_client = None
else:
    print("ℹ️  Foundry not configured — will use built-in email template.")


# ============================================================================
# Create email_notifications Delta table if it doesn't exist
# ============================================================================

spark.sql("""
    CREATE TABLE IF NOT EXISTS email_notifications (
        notification_id   STRING,
        run_id            STRING,
        impact_id         STRING,
        policy_id         STRING,
        alert_id          STRING,
        recipient_email   STRING,
        recipient_name    STRING,
        subject           STRING,
        body_html         STRING,
        body_text         STRING,
        generation_method STRING,
        dispatch_status   STRING,
        dispatch_error    STRING,
        created_at        TIMESTAMP,
        dispatched_at     TIMESTAMP
    ) USING DELTA
""")

print("✅ email_notifications table ready.")

---
## 📥 Step 1 — Load Unnotified Impact Events

In [ ]:
print("📥 Loading unnotified weather impact events from lakehouse...")

try:
    pending_df = spark.sql(f"""
        SELECT *
        FROM   weather_impact_events
        WHERE  impact_status = 'pending_notification'
          AND  risk_score   >= {config.min_risk_score}
        ORDER BY risk_score DESC, created_at DESC
        LIMIT  {config.max_events_per_run}
    """)
    pending = [row.asDict() for row in pending_df.collect()]
    print(f"  Pending impact events to process: {len(pending)}")
except Exception as e:
    print(f"  ❌ Could not query weather_impact_events: {e}")
    print("     Ensure weather_alert_policy_impact.ipynb has been run first.")
    pending = []

if not pending:
    print("\nℹ️  No pending impact events. Either:")
    print("   • Run weather_alert_policy_impact.ipynb first to populate impact events, or")
    print("   • All current events have already been notified.")

if pending:
    display(pending_df.select(
        "impact_id", "policy_id", "business_name", "city", "state",
        "alert_event", "alert_severity", "risk_score", "contact_email"
    ))

---
## 🤖 Steps 2–3 — Orchestrator Agent → Fabric Data Agent → Email Generation

For each pending impact event, the **Orchestrator Agent** will:

1. **Call the Fabric Data Agent** (as a registered tool/action) to retrieve enriched context:
   - Full policy details, claim history, prior outages, current policy status
2. **Compose a professional, personalised email** using that context plus the alert details

If Foundry is not configured, a rule-based template is used instead.

In [ ]:
# ============================================================================
# Helper: extract text from a Responses-API response object
# ============================================================================

def extract_response_text(response) -> str:
    """Pull plain text out of a Foundry Responses API response object."""
    text_parts = []
    try:
        for item in response.output:
            if hasattr(item, "content"):
                for block in item.content:
                    if hasattr(block, "text"):
                        text_parts.append(block.text)
            elif hasattr(item, "text"):
                text_parts.append(item.text)
    except Exception:
        pass
    return "\n".join(text_parts).strip()


# ============================================================================
# Helper: rule-based email template (fallback when Foundry is unavailable)
# ============================================================================

def generate_template_email(imp: Dict[str, Any]) -> Dict[str, str]:
    """
    Generate a professional policy impact notification email from a
    hard-coded template. Used as fallback when Foundry is not configured.
    """
    onset_str   = imp.get("alert_onset").strftime("%B %d, %Y at %I:%M %p UTC") if imp.get("alert_onset") else "Immediately"
    expires_str = imp.get("alert_expires").strftime("%B %d, %Y at %I:%M %p UTC") if imp.get("alert_expires") else "Until further notice"

    subject = (
        f"[{config.insurer_name}] Weather Alert — {imp['alert_event']} "
        f"near {imp['city']}, {imp['state']} | Policy {imp['policy_id']}"
    )

    body_text = textwrap.dedent(f"""
        Dear {imp['business_name']},

        This is an important notification from {config.insurer_name} regarding your
        Business Interruption Insurance policy ({imp['policy_id']}).

        A {imp['alert_severity']} weather alert has been issued in your area that
        may impact your business operations.

        ALERT DETAILS
        ─────────────────────────────────────────────────────────────────
        Alert Type:   {imp['alert_event']}
        Severity:     {imp['alert_severity']}
        Urgency:      {imp['alert_urgency']}
        Affected Area:{imp['alert_area_desc']}
        Onset:        {onset_str}
        Expires:      {expires_str}
        Distance:     Approximately {imp['distance_km']} km from your registered location

        WHAT THIS MEANS FOR YOUR POLICY
        ─────────────────────────────────────────────────────────────────
        Your policy is designed to respond automatically to qualifying power
        outages. If this weather event causes an outage that exceeds your
        threshold of {imp['threshold_minutes']} minutes, a claim will be
        processed automatically — no action is required from you.

        Your coverage summary:
          • Policy ID:          {imp['policy_id']}
          • Outage Threshold:   {imp['threshold_minutes']} minutes
          • Hourly Rate:        ${imp['hourly_rate']:,.2f}/hour
          • Maximum Payout:     ${imp['max_payout']:,.2f}

        RECOMMENDED STEPS
        ─────────────────────────────────────────────────────────────────
        1. Review the NWS alert at https://alerts.weather.gov
        2. Ensure your contact information is current in our portal
        3. Document any business disruptions for your records
        4. Claims are processed automatically — no claim form needed

        CONTACT US
        ─────────────────────────────────────────────────────────────────
        Phone:  {config.insurer_support_phone}
        Email:  {config.insurer_support_email}
        Portal: {config.insurer_portal_url}

        This notification was generated automatically based on active NWS
        weather alerts and your registered policy location.

        Sincerely,
        {config.insurer_name} — Risk Monitoring Team
    """).strip()

    # Minimal HTML wrapper
    body_html = (
        "<html><body style='font-family:Arial,sans-serif;max-width:680px;margin:auto;'>"
        + body_text.replace("\n", "<br>")
        + "</body></html>"
    )

    return {"subject": subject, "body_text": body_text, "body_html": body_html}


print("✅ Email generation helpers ready.")

In [ ]:
# ============================================================================
# Core: Orchestrator Agent → Fabric Data Agent → Email
# ============================================================================

def call_orchestrator_for_email(imp: Dict[str, Any]) -> Dict[str, str]:
    """
    Invoke the Orchestrator Agent (Foundry Responses API).

    The Orchestrator Agent's system prompt should instruct it to:
      1. Call its registered Fabric Data Agent action with a natural-language
         question to retrieve enriched policy/claim context from the lakehouse.
      2. Use that context along with the provided alert + policy data to
         draft a professional, empathetic, and accurate email notification.

    Returns a dict with keys: subject, body_text, body_html
    Falls back to generate_template_email() on any error.
    """
    if not foundry_client or not orchestrator_agent:
        return generate_template_email(imp)

    onset_str   = imp.get("alert_onset").isoformat()  if imp.get("alert_onset")   else "unknown"
    expires_str = imp.get("alert_expires").isoformat() if imp.get("alert_expires") else "unknown"

    # ── Prompt: give the Orchestrator all alert + policy facts, and ask it
    # to call the Fabric Data Agent for enrichment before composing the email.
    user_prompt = f"""
You are an insurance operations AI for {config.insurer_name}.

A severe weather alert has been detected near one of our policyholders.
Your task:

STEP 1 — DATA ENRICHMENT (use the Fabric Data Agent tool)
Call the Fabric Data Agent to answer:
"For policy {imp['policy_id']} ({imp['business_name']}), provide:
  - Full policy details (status, effective date, coverage)
  - Number and outcome of prior claims in the last 12 months
  - Any active claims or pending payouts
  - The most recent outage event associated with this policy, if any"

STEP 2 — EMAIL COMPOSITION
Using the enriched data from Step 1 PLUS the alert details below,
compose a professional, clear, and empathetic email notification.

The email must:
  • Open with a warm but urgent subject line
  • Acknowledge the specific weather threat in plain language
  • Reference the policyholder by business name
  • Explain exactly what triggers their parametric coverage
  • Personalise based on their claim history (if any prior claims: reassure;
    if first potential event: explain the automatic process clearly)
  • List 3–4 recommended preparation steps specific to the alert type
  • Close with contact information and a confident, supportive tone
  • NOT promise a specific payout — only explain the coverage mechanism

ALERT DETAILS:
  Alert Type:    {imp['alert_event']}
  Severity:      {imp['alert_severity']} | Urgency: {imp['alert_urgency']} | Certainty: {imp.get('alert_certainty','Unknown')}
  Headline:      {imp.get('alert_headline', '')}
  Affected Area: {imp.get('alert_area_desc', '')}
  Onset:         {onset_str}
  Expires:       {expires_str}

POLICYHOLDER DETAILS (from weather impact event):
  Policy ID:         {imp['policy_id']}
  Business Name:     {imp['business_name']}
  Business Type:     {imp['business_type']}
  City, State:       {imp['city']}, {imp['state']}
  Contact Email:     {imp['contact_email']}
  Outage Threshold:  {imp['threshold_minutes']} minutes
  Hourly Rate:       ${imp['hourly_rate']:,.2f}/hour
  Max Payout:        ${imp['max_payout']:,.2f}
  Distance from alert centroid: {imp['distance_km']} km
  Risk Score:        {imp['risk_score']} (0–1 scale)

INSURER CONTACT INFO:
  Name:    {config.insurer_name}
  Support: {config.insurer_support_phone} | {config.insurer_support_email}
  Portal:  {config.insurer_portal_url}

OUTPUT FORMAT — return a JSON object with exactly these keys:
{{
  "subject": "<email subject line>",
  "body_text": "<plain-text email body>",
  "body_html": "<HTML email body with inline styles, no external CSS>"
}}

Return ONLY the JSON object. No markdown fences, no preamble.
"""

    try:
        response = foundry_client.agents.responses.create(
            agent_id = config.orchestrator_agent_id,
            input    = [{"role": "user", "content": user_prompt}],
            # The Orchestrator Agent must have the Fabric Data Agent registered
            # as a connected agent action in the Foundry portal for tool-calling
            # to work. The SDK will handle the tool_call / tool_result round-trip
            # automatically when the agent decides to call it.
        )

        raw_text = extract_response_text(response).strip()

        if not raw_text:
            print(f"  ⚠️  No text in Orchestrator response — using template")
            return generate_template_email(imp)

        # Strip markdown code fences if present
        if raw_text.startswith("```"):
            raw_text = raw_text.split("\n", 1)[1].rsplit("```", 1)[0].strip()

        parsed = json.loads(raw_text)

        # Validate required keys
        for key in ("subject", "body_text", "body_html"):
            if key not in parsed:
                raise ValueError(f"Missing key '{key}' in agent response")

        print(f"  ✅ Orchestrator generated email for {imp['business_name']}")
        return parsed

    except json.JSONDecodeError as e:
        print(f"  ⚠️  JSON parse error ({imp['policy_id']}): {e} — using template")
        return generate_template_email(imp)
    except Exception as e:
        print(f"  ⚠️  Orchestrator error ({imp['policy_id']}): {e} — using template")
        return generate_template_email(imp)


print("✅ Orchestrator call function ready.")

In [ ]:
# ============================================================================
# Main loop: process each pending impact event
# ============================================================================

if not pending:
    print("ℹ️  No pending impact events to process. Skipping email generation.")
else:
    print(f"🤖 Processing {len(pending)} impact event(s)...")
    print(f"   Using: {'Orchestrator → Fabric Data Agent' if FOUNDRY_ENABLED and FABRIC_DATA_AGENT_ENABLED else 'Orchestrator Agent only' if FOUNDRY_ENABLED else 'Rule-based template'}\n")

    notification_records: List[Dict[str, Any]] = []

    for i, imp in enumerate(pending, 1):
        print(f"[{i}/{len(pending)}] {imp['business_name']} ({imp['city']}, {imp['state']}) — {imp['alert_event']}")

        email = call_orchestrator_for_email(imp)
        generation_method = (
            "orchestrator+fabric_data_agent" if FOUNDRY_ENABLED and FABRIC_DATA_AGENT_ENABLED
            else "orchestrator_agent" if FOUNDRY_ENABLED
            else "rule_based_template"
        )

        notification_records.append({
            "notification_id":   f"NOTIF-{RUN_ID}-{uuid.uuid4().hex[:6].upper()}",
            "run_id":            RUN_ID,
            "impact_id":         imp["impact_id"],
            "policy_id":         imp["policy_id"],
            "alert_id":          imp["alert_id"],
            "recipient_email":   imp["contact_email"],
            "recipient_name":    imp["business_name"],
            "subject":           email["subject"],
            "body_html":         email["body_html"],
            "body_text":         email["body_text"],
            "generation_method": generation_method,
            "dispatch_status":   "pending",
            "dispatch_error":    None,
            "created_at":        now_utc,
            "dispatched_at":     None,
        })

        # Brief pause to avoid rate-limiting on the Foundry endpoint
        if FOUNDRY_ENABLED and i < len(pending):
            time.sleep(1)

    print(f"\n  ✅ Generated {len(notification_records)} email(s).")

---
## 💾 Step 4 — Persist Emails to Delta

In [ ]:
if not pending:
    print("ℹ️  Nothing to save.")
else:
    notif_schema = StructType([
        StructField("notification_id",   StringType()),
        StructField("run_id",            StringType()),
        StructField("impact_id",         StringType()),
        StructField("policy_id",         StringType()),
        StructField("alert_id",          StringType()),
        StructField("recipient_email",   StringType()),
        StructField("recipient_name",    StringType()),
        StructField("subject",           StringType()),
        StructField("body_html",         StringType()),
        StructField("body_text",         StringType()),
        StructField("generation_method", StringType()),
        StructField("dispatch_status",   StringType()),
        StructField("dispatch_error",    StringType()),
        StructField("created_at",        TimestampType()),
        StructField("dispatched_at",     TimestampType()),
    ])

    notif_df = spark.createDataFrame(notification_records, schema=notif_schema)
    notif_df.write.format("delta").mode("append").saveAsTable("email_notifications")
    print(f"✅ Saved {len(notification_records)} email record(s) to Delta table: email_notifications")

---
## 🚀 Step 5 — [Optional] Dispatch Emails via Webhook

In [ ]:
# ============================================================================
# Optional: POST email payloads to a Logic App / Power Automate webhook
# or any HTTP endpoint that accepts JSON and dispatches emails.
#
# Expected webhook payload shape:
#   {
#     "to":      "recipient@example.com",
#     "subject": "...",
#     "html":    "<html>...",
#     "text":    "...",
#     "from":    "noreply@parametricshield.com",
#     "metadata": { "notification_id": "...", "policy_id": "...", "impact_id": "..." }
#   }
# ============================================================================

if not pending:
    print("ℹ️  Nothing to dispatch.")
elif not DISPATCH_ENABLED:
    print("ℹ️  Email dispatch webhook not configured.")
    print("   Set config.email_dispatch_webhook to a Logic App / Power Automate URL to send emails.")
    print("   Emails are saved to the email_notifications Delta table for review.")
else:
    print(f"🚀 Dispatching {len(notification_records)} email(s) via webhook...\n")
    dispatched, dispatch_failed = 0, 0

    for notif in notification_records:
        payload = {
            "to":      notif["recipient_email"],
            "subject": notif["subject"],
            "html":    notif["body_html"],
            "text":    notif["body_text"],
            "from":    f"noreply@{config.insurer_support_email.split('@')[1]}",
            "metadata": {
                "notification_id": notif["notification_id"],
                "policy_id":       notif["policy_id"],
                "impact_id":       notif["impact_id"],
                "run_id":          notif["run_id"],
            }
        }
        try:
            resp = requests.post(
                config.email_dispatch_webhook,
                json=payload,
                timeout=15,
            )
            resp.raise_for_status()
            notif["dispatch_status"] = "dispatched"
            notif["dispatched_at"]   = datetime.now(timezone.utc).replace(tzinfo=None)
            dispatched += 1
            print(f"  ✅ Dispatched → {notif['recipient_email']} ({notif['policy_id']})")
        except Exception as e:
            notif["dispatch_status"] = "failed"
            notif["dispatch_error"]  = str(e)[:500]
            dispatch_failed += 1
            print(f"  ❌ Failed    → {notif['recipient_email']} ({notif['policy_id']}): {e}")

    print(f"\n  Dispatched: {dispatched}  |  Failed: {dispatch_failed}")

    # Update dispatch status in Delta
    if dispatched or dispatch_failed:
        update_schema = StructType([
            StructField("notification_id", StringType()),
            StructField("dispatch_status", StringType()),
            StructField("dispatch_error",  StringType()),
            StructField("dispatched_at",   TimestampType()),
        ])
        update_df = spark.createDataFrame(
            [{"notification_id": n["notification_id"],
              "dispatch_status": n["dispatch_status"],
              "dispatch_error":  n.get("dispatch_error"),
              "dispatched_at":   n.get("dispatched_at")} for n in notification_records],
            schema=update_schema,
        )
        update_df.createOrReplaceTempView("dispatch_updates")
        spark.sql("""
            MERGE INTO email_notifications AS target
            USING dispatch_updates AS source
            ON target.notification_id = source.notification_id
            WHEN MATCHED THEN UPDATE SET
                target.dispatch_status = source.dispatch_status,
                target.dispatch_error  = source.dispatch_error,
                target.dispatched_at   = source.dispatched_at
        """)
        print("✅ Dispatch status updated in email_notifications.")

---
## ✅ Step 6 — Mark Impact Events as Notified

In [ ]:
if not pending:
    print("ℹ️  Nothing to mark.")
else:
    processed_impact_ids = [imp["impact_id"] for imp in pending]
    ids_literal = ", ".join(f"'{iid}'" for iid in processed_impact_ids)

    try:
        spark.sql(f"""
            UPDATE weather_impact_events
            SET    impact_status = 'notified'
            WHERE  impact_id IN ({ids_literal})
        """)
        print(f"✅ Marked {len(processed_impact_ids)} impact event(s) as 'notified' in weather_impact_events.")
    except Exception as e:
        print(f"⚠️  Could not update impact_status: {e}")
        print("   Impact events remain in 'pending_notification' status — they will be reprocessed next run.")

---
## 📊 Step 7 — Summary Dashboard

In [ ]:
print("=" * 70)
print(f"RUN SUMMARY — Run ID: {RUN_ID}")
print("=" * 70)
print(f"  Email generation: {'Orchestrator + Fabric Data Agent' if FOUNDRY_ENABLED and FABRIC_DATA_AGENT_ENABLED else 'Orchestrator Agent' if FOUNDRY_ENABLED else 'Rule-based template'}")
print(f"  Impact events processed: {len(pending)}")

if pending:
    gen_counts = {}
    for n in notification_records:
        m = n["generation_method"]
        gen_counts[m] = gen_counts.get(m, 0) + 1
    for method, cnt in gen_counts.items():
        print(f"  Generated via {method}: {cnt}")

print()
print("--- Email Notifications (This Run) ---")
try:
    display(spark.sql(f"""
        SELECT
            notification_id,
            policy_id,
            recipient_name,
            recipient_email,
            subject,
            generation_method,
            dispatch_status,
            created_at
        FROM email_notifications
        WHERE run_id = '{RUN_ID}'
        ORDER BY created_at
    """))
except Exception as e:
    print(f"  Could not display table: {e}")

print()
print("--- Email Notifications (All Time) ---")
try:
    display(spark.sql("""
        SELECT
            generation_method,
            dispatch_status,
            COUNT(*) AS total,
            MIN(created_at) AS first_sent,
            MAX(created_at) AS last_sent
        FROM email_notifications
        GROUP BY generation_method, dispatch_status
        ORDER BY total DESC
    """))
except Exception:
    pass

print()
print("--- Sample Email Preview (most recent) ---")
try:
    sample = spark.sql("""
        SELECT subject, body_text
        FROM   email_notifications
        ORDER BY created_at DESC
        LIMIT  1
    """).collect()
    if sample:
        print(f"\nSUBJECT: {sample[0]['subject']}\n")
        # Print first 60 lines of the body
        body_lines = (sample[0]['body_text'] or "").split("\n")
        for line in body_lines[:60]:
            print(line)
        if len(body_lines) > 60:
            print(f"... [{len(body_lines) - 60} more lines]")
except Exception as e:
    print(f"  Could not preview email: {e}")

# Refresh SQL Endpoint
access_token = mssparkutils.credentials.getToken("https://api.fabric.microsoft.com/")

# Fabric REST API endpoint for refreshing SQL endpoint metadata
workspace_id = "c244c53e-58d3-4055-8dc8-8c79bc0ad4b0"
lakehouse_id = "5166d063-e222-49b7-948b-f029f463cc2b"
refresh_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/lakehouses/{lakehouse_id}/refreshSqlEndpoint"

# === CALL THE REFRESH API ===
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

try:
    response = requests.post(refresh_url, headers=headers)
    if response.status_code == 202:
        print("✅ SQL Endpoint refresh triggered successfully.")
    else:
        print(f"⚠️ Failed to refresh SQL Endpoint. Status: {response.status_code}")
        print(response.text)
except Exception as e:
    print(f"❌ Error calling refresh API: {e}")


print()
print("✅ Notebook complete.")
print("   📋 Emails are stored in the email_notifications Delta table.")
print("   🚀 To send emails, configure config.email_dispatch_webhook with a Logic App URL.")